## PyTorch Tutorial
MILA, December 2018

By Sandeep Subramanian

### Torch Autograd, Define-by-run & Execution Paradigm

Adapted from
1. http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py 
2. http://pytorch.org/docs/master/notes/autograd.html

## Torch Tensors and Gradients

Almost all (differentiable) operations on tensors support gradient computation

In [1]:
import numpy as np
from __future__ import print_function

In [3]:
import torch

### Wrap tensors in a Variable

In [4]:
z = torch.Tensor(5, 3).uniform_(-1, 1)
print(z)

tensor([[-0.6116, -0.4678, -0.3097],
        [ 0.8147, -0.2846,  0.6574],
        [ 0.8518,  0.4801,  0.5343],
        [-0.2408, -0.2342, -0.6451],
        [ 0.6954, -0.9469,  0.4634]])


### Properties of Tensors : Requiring gradients, Volatility, Data & Grad

1. You can access the raw tensor value through the .data attribute
2. Gradient of the loss w.r.t. this variable is *accumulated* into .grad.
3. Stay tuned for requires_grad

In [7]:
print(z.data)
print('Requires Gradient : %s ' % (z.requires_grad))
print('Gradient : %s ' % (z.grad))

tensor([[-0.6116, -0.4678, -0.3097],
        [ 0.8147, -0.2846,  0.6574],
        [ 0.8518,  0.4801,  0.5343],
        [-0.2408, -0.2342, -0.6451],
        [ 0.6954, -0.9469,  0.4634]])
Requires Gradient : False 
Gradient : None 


In [13]:
### Operations on Variables
x = torch.Tensor(5, 3).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
# matrix multiplication
z = torch.mm(x, y)
print(z.size())

torch.Size([5, 5])


## Define-by-run Paradigm

The torch autograd package provides automatic differentiation for all operations on Tensors.

PyTorch's autograd is a reverse mode automatic differentiation system.

Backprop is defined by how your code is run, and that every single iteration can be different.

Other frameworks that adopt a similar approach :

1. Chainer - https://github.com/chainer/chainer
2. DyNet - https://github.com/clab/dynet
3. Tensorflow Eager - https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html

### How autograd encodes execution history


Conceptually, autograd maintains a graph that records all of the operations performed on tensors as you execute your operations. This results in a directed acyclic graph whose leaves are the input variables and roots are the output variables. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

![caption](images/dynamic_graph.gif)

GIF source: https://github.com/pytorch/pytorch

Internally, autograd represents this graph as a graph of Function objects (really expressions), which can be `apply()` ed to compute the result of evaluating the graph. When computing the forwards pass, autograd simultaneously performs the requested computations and builds up a graph representing the function that computes the gradient (the `.grad_fn` attribute of each Variable is an entry point into this graph). When the forwards pass is completed, we evaluate this graph in the backwards pass to compute the gradients.

In [16]:
x = torch.Tensor(5, 3).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
x.requires_grad = True
z = torch.mm(x, y)
print(z.grad_fn)

An important thing to note is that the graph is recreated from scratch at every iteration, and this is exactly what allows for using arbitrary Python control flow statements, that can change the overall shape and size of the graph at every iteration. You don’t have to encode all possible paths before you launch the training - what you run is what you differentiate.

## Getting gradients : `backward()` & `torch.autograd.grad`

In [17]:
x = torch.Tensor(5, 3).uniform_(-1, 1)
y = torch.Tensor(5, 3).uniform_(-1, 1)
x.requires_grad = True
y.requires_grad = True
z = x ** 2 + 3 * y
z.backward(gradient=torch.ones(5, 3))

In [20]:
# eq computes element-wise equality
torch.eq(x.grad, 2 * x)

tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype=torch.uint8)

In [21]:
y.grad

tensor([[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]])

In [23]:
x = torch.Tensor(5, 3).uniform_(-1, 1)
y = torch.Tensor(5, 3).uniform_(-1, 1)
x.requires_grad = True
y.requires_grad = True
z = x ** 2 + 3 * y
dz_dx = torch.autograd.grad(z, x, grad_outputs=torch.ones(5, 3))
dz_dy = torch.autograd.grad(z, y, grad_outputs=torch.ones(5, 3))

## Define-by-run example

### Common Tensor definitions

In [28]:
x = torch.Tensor(5, 3).uniform_(-1, 1)
w = torch.Tensor(3, 10).uniform_(-1, 1)
b = torch.Tensor(10,).uniform_(-1, 1)

### Set all to require grad

In [29]:
x.requires_grad = True
w.requires_grad = True
b.requires_grad = True

### Graph 1 : `wx + b`

In [46]:
o = torch.matmul(x, w) + b
do_dinputs_1 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [47]:
print('Gradients of o w.r.t inputs in Graph 1')
print('do/dx : \n\n %s ' % (do_dinputs_1[0]))
print('do/dw : \n\n %s ' % (do_dinputs_1[1]))
print('do/db : \n\n %s ' % (do_dinputs_1[2]))

Gradients of o w.r.t inputs in Graph 1
do/dx : 

 tensor([[-0.2279, -0.4665,  2.3362],
        [-0.2279, -0.4665,  2.3362],
        [-0.2279, -0.4665,  2.3362],
        [-0.2279, -0.4665,  2.3362],
        [-0.2279, -0.4665,  2.3362]]) 
do/dw : 

 tensor([[-3.0675e+00, -3.0675e+00, -3.0675e+00, -3.0675e+00, -3.0675e+00,
         -3.0675e+00, -3.0675e+00, -3.0675e+00, -3.0675e+00, -3.0675e+00],
        [-1.6079e-03, -1.6079e-03, -1.6079e-03, -1.6079e-03, -1.6079e-03,
         -1.6079e-03, -1.6079e-03, -1.6079e-03, -1.6079e-03, -1.6079e-03],
        [-3.3780e-01, -3.3780e-01, -3.3780e-01, -3.3780e-01, -3.3780e-01,
         -3.3780e-01, -3.3780e-01, -3.3780e-01, -3.3780e-01, -3.3780e-01]]) 
do/db : 

 tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]) 


In [48]:
o = torch.matmul(x, w) + b
loss = ((o - (torch.ones(5, 10) * 10)) ** 2).mean()
dloss_dinputs = torch.autograd.grad(loss, [x, w, b, o])

In [49]:
print('Gradients of loss w.r.t inputs & o in Graph 1')
print('dl/dx : \n\n %s ' % (dloss_dinputs[0]))
print('dl/dw : \n\n %s ' % (dloss_dinputs[1]))
print('dl/db : \n\n %s ' % (dloss_dinputs[2]))
print('dl/do : \n\n %s ' % (dloss_dinputs[3]))

Gradients of loss w.r.t inputs & o in Graph 1
dl/dx : 

 tensor([[ 0.0540,  0.1836, -0.9017],
        [-0.0324,  0.2870, -0.9882],
        [ 0.1716,  0.0405, -0.9905],
        [-0.0060,  0.3200, -0.8747],
        [ 0.0236,  0.1326, -1.0127]]) 
dl/dw : 

 tensor([[ 1.2512,  1.3968,  1.1936,  1.3066,  1.1642,  1.3183,  1.2312,  1.0726,
          1.3070,  1.1774],
        [ 0.0591, -0.0297,  0.0911, -0.0247,  0.1209, -0.0211, -0.0279,  0.0913,
         -0.1499, -0.0396],
        [ 0.1983,  0.1893,  0.2304,  0.0912,  0.1795,  0.1655,  0.1358,  0.1932,
          0.0799,  0.1144]]) 
dl/db : 

 tensor([-2.0644, -2.2232, -2.0357, -2.1252, -2.0168, -2.1010, -1.9994, -1.8325,
        -1.9829, -1.9155]) 
dl/do : 

 tensor([[-0.4046, -0.4251, -0.3849, -0.4394, -0.4090, -0.4098, -0.3903, -0.3556,
         -0.3892, -0.3754],
        [-0.4119, -0.4750, -0.4083, -0.4137, -0.3690, -0.4395, -0.4137, -0.3568,
         -0.4397, -0.3956],
        [-0.4267, -0.4247, -0.4532, -0.4195, -0.4568, -0.4009, -0.39

### Graph 2 : wx / b

In [50]:
o = torch.matmul(x, w) / b
do_dinputs_2 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [51]:
print('Gradients of o w.r.t inputs in Graph 2')
print('do/dx : \n %s ' % (do_dinputs_2[0]))
print('do/dw : \n %s ' % (do_dinputs_2[1]))
print('do/db : \n %s ' % (do_dinputs_2[2]))

Gradients of o w.r.t inputs in Graph 2
do/dx : 
 tensor([[-11.0835, -11.9203, -11.9643],
        [-11.0835, -11.9203, -11.9643],
        [-11.0835, -11.9203, -11.9643],
        [-11.0835, -11.9203, -11.9643],
        [-11.0835, -11.9203, -11.9643]]) 
do/dw : 
 tensor([[ 4.0403e+01,  4.2945e+00,  7.2919e+00,  3.7218e+00,  6.9264e+00,
          2.9755e+01,  1.9412e+01, -4.6716e+00, -5.9660e+00, -2.9877e+01],
        [ 2.1178e-02,  2.2510e-03,  3.8221e-03,  1.9509e-03,  3.6304e-03,
          1.5596e-02,  1.0176e-02, -2.4487e-03, -3.1274e-03, -1.5661e-02],
        [ 4.4493e+00,  4.7292e-01,  8.0301e-01,  4.0985e-01,  7.6276e-01,
          3.2767e+00,  2.1377e+00, -5.1445e-01, -6.5700e-01, -3.2901e+00]]) 
do/db : 
 tensor([ 213.5344,    3.9383,   -6.8415,   -1.4584,   -9.1481,  189.0114,
         -32.2513,   -2.0958,    8.1083, -151.6411]) 


## Gradient buffers: `.backward()` and `retain_graph=True`

1. Calling `.backward()` clears the current computation graph.
2. Once `.backward()` is called, intermediate tensors used in the construction of the graph are removed.
2. This is used implicitly to let PyTorch know when a new graph is to be built for a new minibatch. This is built around the forward and backward pass paradigm.
3. To retain the graph after the backward pass use `loss.backward(retain_graph=True)`. This lets you re-use intermediate tensors (and even their gradients) to potentially compute a secondary loss after the initial gradients are computed. This is useful to implement things like the gradient penalty in WGANs (https://arxiv.org/abs/1704.00028)

In [52]:
o = torch.mm(x, w) + b
loss = ((o - (torch.ones(5, 10) * 10)) ** 2).mean()
loss.backward()

### Call backward again -> <font color='red'>This fails</font>

In [54]:
loss += torch.norm(w)
loss.backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

###  But with `retain_graph=True`

In [58]:
o = torch.mm(x, w) + b
loss = ((o - (torch.ones(5, 10) * 10)) ** 2).mean()
loss.backward(retain_graph=True)

In [59]:
loss += torch.norm(w)
loss.backward()

## <font color='red'>WARNING:</font> Calling `.backward()` multiple times will accumulate gradients into `.grad` and NOT overwrite them.

## Excluding subgraphs from backward: requires_grad=False, .detach() and torch.no_grad()

### `requires_grad=False`

1. If there’s a single input to an operation that requires gradient, its output will also require gradient.

2. Conversely, if all inputs don’t require gradient, the output won’t require it.

3. Backward computation is never performed in the subgraphs, where all Variables didn’t require gradients.

4. This is potentially useful when you have part of a network that is pretrained and not fine-tuned, for example word embeddings or a pretrained imagenet model.

In [63]:
x = torch.Tensor(3, 5).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
z = torch.Tensor(3, 5).uniform_(-1, 1)

In [64]:
# Requires grad assignments
x.requires_grad = False
y.requires_grad = False
z.requires_grad = True

In [65]:
o = x + y
print(' o = x + y requires grad ? : %s ' % (o.requires_grad))
o = x + y + z
print(' o = x + y + z requires grad ? : %s ' % (o.requires_grad))

 o = x + y requires grad ? : False 
 o = x + y + z requires grad ? : True 


### `.detach()`

1. It is possible to detach variables from the graph by calling `.detach()`. 
2. This could lead to disconnected graphs. In which case PyTorch will only backpropagate gradients until the point of disconnection.

In [69]:
x = torch.Tensor(3, 5).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
z = torch.Tensor(3, 5).uniform_(-1, 1)

![caption](images/detach.png)

In [70]:
# Setting gradient requirements
x.requires_grad = True
y.requires_grad = True
z.requires_grad = True

In [71]:
m1 = x + y
m2 = z ** 2
m1 = m1.detach()
m3 = m1 + m2
m3.backward(torch.ones(3, 5))

In [72]:
print('dm3/dx \n\n %s ' % (x.grad))
print('\ndm3/dy \n\n %s ' % (y.grad))
print('\ndm3/dz \n\n %s ' % (z.grad))

dm3/dx 

 None 

dm3/dy 

 None 

dm3/dz 

 tensor([[ 0.5873,  0.3034,  0.2369, -0.2147, -1.0424],
        [ 1.0415, -1.3117, -1.1575, -0.4519,  1.6409],
        [-0.7689,  0.0927,  1.1965,  1.2050, -0.0966]]) 


## Gradients w.r.t intermediate variables in the graph

1. By default, PyTorch all gradient computations w.r.t intermediate nodes in the graph are ad-hoc.

2. This is in the interest of saving memory.

3. To compute gradients w.r.t intermediate variables, use `.retain_grad()` or explicitly compute gradients using `torch.autograd.grad`

4. `.retain_grad()` populates the `.grad` attribute of the tensor while `torch.autograd.grad` returns a tensor that contains the gradients.

In [73]:
x = torch.Tensor(3, 5).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
z = torch.Tensor(3, 5).uniform_(-1, 1)

In [74]:
# Setting gradient requirements
x.requires_grad = True
y.requires_grad = True
z.requires_grad = True

In [75]:
m1 = x + y
m2 = z ** 2
m1.retain_grad()
m2.retain_grad()
m3 = m1 * m2
m3.backward(torch.ones(3, 5))

In [76]:
print('dm3/dm1 \n\n %s ' % (m1.grad))
print('dm3/dm2 \n\n %s ' % (m2.grad))

dm3/dm1 

 tensor([[0.0475, 0.3182, 0.0710, 0.4032, 0.0499],
        [0.3085, 0.0627, 0.1340, 0.0162, 0.3985],
        [0.0622, 0.0447, 0.0616, 0.0132, 0.1004]]) 
dm3/dm2 

 tensor([[ 0.4285,  0.1395, -0.2023,  0.0714,  0.8476],
        [ 1.7018, -0.9725, -0.9733, -0.2363,  0.9779],
        [ 1.0983,  0.9697,  0.1050, -0.7002, -0.4050]]) 


### In place operations on variables in a graph

source: http://pytorch.org/docs/master/notes/autograd.html

In place operations are suffixed by `_` ex: `log_`, `uniform_` etc.

1. Supporting in-place operations in autograd is difficult and PyTorch discourages their use in most cases.

2. Autograd’s aggressive buffer freeing and reuse makes it very efficient and there are very few occasions when in-place operations actually lower memory usage by any significant amount. Unless you’re operating under heavy memory pressure, you might never need to use them.

### There are two main reasons that limit the applicability of in-place operations:

(a) Overwriting values required to compute gradients. This is why variables don’t support `log_`. Its gradient formula requires the original input, and while it is possible to recreate it by computing the inverse operation, it is numerically unstable, and requires additional work that often defeats the purpose of using these functions.

(b) Every in-place operation actually requires the implementation to rewrite the computational graph. Out-of-place versions simply allocate new objects and keep references to the old graph, while in-place operations, require changing the creator of all inputs to the Function representing this operation. This can be tricky, especially if there are many Variables that reference the same storage (e.g. created by indexing or transposing), and in-place functions will actually raise an error if the storage of modified inputs is referenced by any other Variable.
In-place correctness checks

## Second and higher order derivatives

### Computing gradients w.r.t gradients

1. `o = xy + z ** 2`
2. `l = o ** 3 + do_dz`

In [84]:
x = torch.Tensor(5, 3).uniform_(-1, 1)
y = torch.Tensor(3, 5).uniform_(-1, 1)
z = torch.Tensor(5, 5).uniform_(-1, 1)

In [85]:
# Setting gradient requirements
x.requires_grad = True
y.requires_grad = True
z.requires_grad = True

In [86]:
o = torch.mm(x, y) + z ** 2
# if create_graph=False then the resulting gradient is volatile and cannot be used further to compute a second loss.
do_dz = torch.autograd.grad(o, z, grad_outputs=torch.ones(5, 5), retain_graph=True, create_graph=True)
print('do/dz \n\n : %s ' % (do_dz[0]))
l = o ** 3 + do_dz[0]
dl_dz = torch.autograd.grad(l, z, grad_outputs=torch.ones(5, 5))
print('dl/dz \n\n : %s ' % (dl_dz[0]))

do/dz 

 : tensor([[ 1.1486, -0.5590,  0.1641, -1.1943,  1.7544],
        [-0.8638,  1.4439, -1.3479,  1.1914, -1.4932],
        [-1.7980, -1.8647,  0.7458, -1.5407, -0.6826],
        [-1.6410, -1.4988, -0.6686,  1.0944,  0.9396],
        [ 0.2901,  1.2748,  1.0154,  0.4187,  0.7825]], grad_fn=<MulBackward0>) 
dl/dz 

 : tensor([[ 2.3091,  1.8875,  2.0071,  1.1295,  9.3688],
        [ 1.9759,  2.0020,  0.1605,  2.6641, -1.2088],
        [-0.6832,  0.2437,  2.0689,  1.2481,  1.5517],
        [ 1.3195, -0.5342,  0.0706,  2.2338,  2.2560],
        [ 2.0005,  2.7764,  3.9440,  5.3283,  2.3846]]) 
